## HP Laptops Web Scraping

HP Inc. is an American multinational information technology company headquartered in Palo Alto, California, that develops personal computers (PCs), printers and related supplies, as well as 3D printing solutions.

## Importing Required Libraries

In [36]:
# load the required libraries

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup

import re
#import time


In [14]:
url='https://www.hp.com/in-en/shop/catalogsearch/result/index/?p=1&product_list_limit=24&q=laptop'

In [15]:
page=requests.get(url)

In [16]:
page.status_code

200

In [ ]:
page.text

In [6]:
pagecontent=page.text

In [ ]:
page.headers

In [ ]:
page.text

In [ ]:
page.content

In [17]:
# get the HTML content from the page
pagecontent = page.text

In [ ]:
pagecontent

In [18]:
requests.get(url).text

'<!doctype html>\n<html lang="en-in">\n    <head >\n        \n<script>\n    var BASE_URL = \'https://www.hp.com/in-en/shop/\';\n    var require = {\n        "baseUrl": "https://in-media.apjonlinecdn.com/static/version1700000835/frontend/HPOLS/stellar/en_US"\n    };\n</script>\n        <meta charset="utf-8"/>\n<meta name="description" content="HP India’s most complete online store for Laptops, PCs, Tablets, Monitors, Printers, Inks & Toners, Workstations, Accessories and more! Largest selection for HP brands at lowest price. Payment options - COD, EMI, Credit card, Debit card &amp; more. Buy Now!"/>\n<meta name="keywords" content="HP Online Shopping in India, HP online Shopping store, HP Online Shopping Site, HP Buy Online, Shop HP Online, HP Online Shopping, HP"/>\n<meta name="robots" content="NOINDEX,NOFOLLOW"/>\n<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no"/>\n<meta name="format-detection" content="teleph

In [19]:
s = BeautifulSoup(pagecontent)

In [20]:
prices=[]
productname=[]
original_price=[]
specs=[]
gst_price=[]
price_saved=[]
offer_per=[]
for i in range(1,16):
    for x in s.find_all('div', attrs={'class':'product details product-item-details'}):
        price = int(re.sub(r'[₹,]','',re.findall(r'[₹0-9,]+',x.find('div', attrs={'class':'price-box price-final_price'}).text)[0]))
        title = re.sub(r'[\n ]','',x.find('h2',attrs={'class':'plp-h2-title stellar-title__small'}).text)
        orig_price = int(re.sub(r'[₹,]','',re.findall(r'[₹0-9,]+',x.find('div',attrs={'class':'suggest-retail-price simple'}).text)[0]))
        spec = re.sub(r'[\n]','',x.find('div',attrs={'class':'product-desc-features stellar-body__small'}).text)
        gst = int(re.sub(r'[₹,]','',re.findall(r'[₹0-9,]+',x.find('div', attrs={'class':'price-box price-final_price'}).text)[1]))
        offer = int(re.sub(r'[₹,]','',re.findall(r'[₹0-9,]+',x.find('div',attrs={'class':'suggest-retail-price simple'}).text)[1]))
        offerper = int(re.sub(r'[₹,]','',re.findall(r'[₹0-9,]+',x.find('div',attrs={'class':'suggest-retail-price simple'}).text)[2]))
        if price==None:
            prices.append(np.NaN)
        else:
            prices.append(price)
        if title==None:
            productname.append(np.NaN)
        else:
            productname.append(title)
        if orig_price==None:
            original_price.append(np.NaN)
        else:
            original_price.append(orig_price)
        if spec==None:
            specs.append(np.NaN)
        else:
            specs.append(spec)
        if gst==None:
            gst_price.append(np.NaN)
        else:
            gst_price.append(gst)
        if offer==None:
            price_saved.append(np.NaN)
        else:
            price_saved.append(offer)
        if offerper==None:
            offer_per.append(np.NaN)
        else:
            offer_per.append(offerper)   

In [21]:
print(len(productname)) 
print(len(prices)) 
print(len(original_price)) 
print(len(specs)) 
print(len(gst_price)) 
print(len(price_saved))
print(len(offer_per)) 

360
360
360
360
360
360
360


In [22]:
df = pd.DataFrame({'ProductName':productname, 'Features':specs, 'Price':prices,  'OriginalPrice':original_price, 'GST':gst_price, 'PriceSaved':price_saved, 'OfferPercenatge':offer_per})

In [23]:
def extract_processor(x):
    return re.findall(r'[A-z0-9-®™ ]+(?:P|p)rocessor',x)[0]

In [24]:
df['Processor'] = df['Features'].apply(extract_processor)

In [25]:
def extract_RAM(x):
    return re.findall(r'[0-9]{1,2}\sGB',x)[0]

In [26]:
df['RAM(GB)']=df['Features'].apply(extract_RAM)

In [27]:
def extract_Hard_drive(x):
    return re.findall(r'[0-9]{1,3}\s?(?:GB|TB)',x)[1]

In [28]:
df['Hard_Drive(GB)']=df['Features'].apply(extract_Hard_drive)

In [29]:
def extract_screen_size(x):
    return re.findall(r'[0-9.]+ cm',x)[0]

In [30]:
df['ScreenSize']=df['Features'].apply(extract_screen_size)

In [31]:
def extract_os(x):
    return re.findall(r'Windows [0-9]+ (?:Pro|Home Single Language|Home)',x)[0]

In [32]:
df['OS']=df['Features'].apply(extract_os)

In [37]:
df.head()

,ProductName,Features,Price,OriginalPrice,GST,PriceSaved,OfferPercenatge,Processor,RAM(GB),Hard_Drive(GB),ScreenSize,OS,ProductID
0,HPPavilionPlus35.6cmLaptop14-ew0154TU-Blue,13th Generation Intel® Core™ i7 processorWindo...,114999,132408,17542,17409,13,13th Generation Intel® Core™ i7 processor,16 GB,1 TB,35.6 cm,Windows 11 Pro,92U78PA
1,HPPavilionPlus35.6cmLaptop14-ey0789AU-Silver,AMD Ryzen™ 7 processorWindows 11 Home35.6 cm (...,91999,105926,14034,13927,13,AMD Ryzen™ 7 processor,16 GB,1TB,35.6 cm,Windows 11 Home,92U77PA
2,HPPavilion35.6cmx3602-in-1Laptop14-ek1009TU-Gold,13th Generation Intel® Core™ i5 processorWindo...,83999,92746,12813,8747,9,13th Generation Intel® Core™ i5 processor,16 GB,1 TB,35.6 cm,Windows 11 Home,7N759PA
3,HPEnvy33.8cmx3602-in-1LaptopOLED13-bf0141TU-Si...,12th Gen Intel® Evo™ Powered by Core™ i7 proce...,109999,121629,16780,11630,9,12th Gen Intel® Evo™ Powered by Core™ i7 proce...,16 GB,512 GB,33.8 cm,Windows 11 Home,81B47PA
4,HPPavilion35.6cmLaptop14-ec1003AU,AMD Ryzen™ 5-5625U ProcessorWindows 11 Home35....,52999,61022,8085,8023,13,AMD Ryzen™ 5-5625U Processor,8 GB,512 GB,35.6 cm,Windows 11 Home,689H4PA


In [164]:
def extract_capacity(x):
    if x is None:
        return None
    elif 'TB' in x:
        return float(re.findall(r'[0-9]+', x)[0])*1024
    else:
        return float(re.findall(r'[0-9]+', x)[0])

In [165]:
df['Hard_Drive(GB)'] = df['Hard_Drive(GB)'].apply(extract_capacity)

In [167]:
df['RAM(GB)'] = df['RAM(GB)'].apply(lambda x: int(re.findall(r'[0-9]+ GB',x)[0][:-3]))

In [95]:
df['ScreenSize'] = df['ScreenSize'].apply(lambda x: float(re.findall(r'[0-9.]+ cm',x)[0][:-3]))

In [53]:
import pickle
with open('HP_Laptop.pkl','wb') as f:
    pickle.dump(df,f)

In [192]:
df['AdditionalFeatures']=df['Features'].apply(lambda x:re.findall(r'(?:D|d)rive\s?[\S ]+ ',x)[0][5:])

In [181]:
df['HardDrive']=df['HardDrive'].apply(lambda x:int(x))

In [34]:
Product_id=[]
for i in range(1,16):
    for x in s.find_all('div', attrs={'class':'product details product-item-details'}):
        pid = x.find('div',attrs={'class':'product-sku stellar-body__extra-small'})
        if pid==None:
            Product_id.append(np.NaN)
        else:
            Product_id.append(pid)

In [35]:
df['ProductID']=Product_id

In [36]:
df['ProductID']=df['ProductID'].apply(lambda x:"".join(x))

In [194]:
order=['ProductID','ProductName','Processor','RAM','HardDrive','OS','ScreenSize','OriginalPrice','Price','GST','PriceSaved','OfferPercentage','AdditionalFeatures']

In [195]:
df=df[order]

In [149]:
df[['OriginalPrice','Price','PriceSaved','GST']].corr()

,OriginalPrice,Price,PriceSaved,GST
OriginalPrice,1.000000,0.991977,0.576109,0.991976
Price,0.991977,1.000000,0.468155,1.000000
PriceSaved,0.576109,0.468155,1.000000,0.468149
GST,0.991976,1.000000,0.468149,1.000000


In [41]:
corr_max = df.corr(numeric_only=True)

In [50]:
import pickle
with open('HP_df.pkl','wb') as f:
    pickle.dump(df,f)

In [51]:
import pickle
with open("HP_df.pkl","rb") as f:
    hello_lap = pickle.load(f)

In [52]:
df=hello_lap.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ProductName      360 non-null    object 
 1   Processor        360 non-null    object 
 2   RAM              360 non-null    int64  
 3   HardDrive        360 non-null    float64
 4   OS               360 non-null    object 
 5   ScreenSize       360 non-null    object 
 6   Features         360 non-null    object 
 7   OriginalPrice    360 non-null    int64  
 8   GST              360 non-null    int64  
 9   OfferPercentage  360 non-null    int64  
 10  PriceSaved       360 non-null    int64  
 11  Price            360 non-null    int64  
 12  ProductID        360 non-null    object 
dtypes: float64(1), int64(6), object(6)
memory usage: 36.7+ KB


In [40]:
df['HardDrive']=df['HardDrive'].apply(lambda x:int(x))

In [9]:
df['Price'].sort_values(ascending=False)

176    136999
224    136999
56     136999
32     136999
104    136999
        ...  
290     52999
338     52999
50      52999
314     52999
266     52999
Name: Price, Length: 360, dtype: int64

In [14]:
df.loc[df.Price==df.Price.max(),:].head(1)

,ProductName,Processor,RAM,HardDrive,OS,ScreenSize,Features,OriginalPrice,GST,OfferPercentage,PriceSaved,Price,ProductID
8,HPEnvy39.6cmx3602-in-1LaptopOLED15-fe0014TX,13th Generation Intel® Core™ i7 processor,16,1024.0,Windows 11 Home,39.6 cm,"Backlit Keyboard, Wide Vision 5MP IR camera, B...",149588,20898,8,12589,136999,8C4R6PA


In [15]:
df.loc[df.Price==df.Price.min(),:].head(1)

,ProductName,Processor,RAM,HardDrive,OS,ScreenSize,Features,OriginalPrice,GST,OfferPercentage,PriceSaved,Price,ProductID
2,HPPavilion35.6cmLaptop14-ec1003AU,AMD Ryzen™ 5-5625U Processor,8,512.0,Windows 11 Home,35.6 cm,"Backlit keyboard, Wide Vision 720p HD camera, ...",61022,8085,13,8023,52999,689H4PA


In [16]:
def extract_processor_type(x):
    if 'Intel' in x:
        return 'Intel'
    elif 'AMD' in x:
        return 'AMD'

In [18]:
df['Processor_Type']=df['Processor'].apply(extract_processor_type)

In [27]:
df.rename(columns={'Additional Features':'Additional_Features'},inplace=True)

In [28]:
order=['ProductName','Processor','RAM','HardDrive','OS','ScreenSize','Additional_Features','Processor_Type','OriginalPrice','GST','OfferPercentage','PriceSaved','Price','ProductID']

In [29]:
df=df[order]

In [30]:
df.head()

,ProductName,Processor,RAM,HardDrive,OS,ScreenSize,Additional_Features,Processor_Type,OriginalPrice,GST,OfferPercentage,PriceSaved,Price,ProductID
0,HPPavilionPlus35.6cmLaptop14-ew0154TU-Blue,13th Generation Intel® Core™ i7 processor,16,1024.0,Windows 11 Pro,35.6 cm,"Backlit keyboard, Wide Vision 5MP IR camera, B...",Intel,132408,17542,13,17409,114999,92U78PA
1,HPPavilionPlus35.6cmLaptop14-ey0789AU-Silver,AMD Ryzen™ 7 processor,16,1024.0,Windows 11 Home,35.6 cm,"Backlit keyboard, Wide Vision 5MP IR camera, B...",AMD,105926,14034,13,13927,91999,92U77PA
2,HPPavilion35.6cmLaptop14-ec1003AU,AMD Ryzen™ 5-5625U Processor,8,512.0,Windows 11 Home,35.6 cm,"Backlit keyboard, Wide Vision 720p HD camera, ...",AMD,61022,8085,13,8023,52999,689H4PA
3,HPPavilion35.6cmx3602-in-1Laptop14-ek1009TU-Gold,13th Generation Intel® Core™ i5 processor,16,1024.0,Windows 11 Home,35.6 cm,"Backlit keyboard, Fingerprint reader, True Vis...",Intel,92746,12813,9,8747,83999,7N759PA
4,HPEnvy33.8cmx3602-in-1LaptopOLED13-bf0141TU-Si...,12th Gen Intel® Evo™ Powered by Core™ i7 proce...,16,512.0,Windows 11 Home,33.8 cm,"Backlit Keyboard, True Vision 5MP IR camera, B...",Intel,121629,16780,9,11630,109999,81B47PA
